<a href="https://colab.research.google.com/github/abudubai16/NLP-using-MoE/blob/main/MoE_NLP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an implementation of Mixture of Experts using Switch Transformers based off of the following research paper (https://arxiv.org/pdf/2101.03961)

I implented a Switching 'Mixture of Experts' layer. Then I created a new torch.nn.TransformerEncoderLayer using inheritance to overwrite pytorch's feedforward section with the new layer I mentioned before. The rest of the code is a standard transformer used of word prediction.

## Downloading required libraries

In [1]:
! pip install portalocker --q
! pip install datasets --q --q
! pip install positional-encodings[pytorch] --q --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
! pip uninstall JAX

Found existing installation: jax 0.4.33
Uninstalling jax-0.4.33:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/jax-0.4.33.dist-info/*
    /usr/local/lib/python3.10/dist-packages/jax/*
Proceed (Y/n)? y
  Successfully uninstalled jax-0.4.33


## Importing libraries

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
from positional_encodings.torch_encodings import PositionalEncoding1D
from datasets import load_dataset, concatenate_datasets

from transformers import RobertaTokenizer
from tokenizers import Tokenizer

import os
import time
import math

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Downloading the dataset

In [5]:
if not os.path.exists('/content/wikitext'):
  ! git clone 'https://huggingface.co/datasets/wikitext' --q

In [6]:
dataset = load_dataset('parquet', data_files={
    'train1':'/content/wikitext/wikitext-103-v1/train-00000-of-00002.parquet',
    'train2': '/content/wikitext/wikitext-103-v1/train-00001-of-00002.parquet',
    'val':'/content/wikitext/wikitext-103-v1/validation-00000-of-00001.parquet',
    'test':'/content/wikitext/wikitext-103-v1/test-00000-of-00001.parquet'
})

# Dataset for training the model
train_ds = concatenate_datasets([
    dataset['train1'],
      dataset['train2'],
      dataset['test']
])['text']

# Dataset for creating the tokenizer
stacked = concatenate_datasets([
      dataset['train1'],
      dataset['train2'],
      dataset['val'],
      dataset['test']
  ])

Generating train1 split: 0 examples [00:00, ? examples/s]

Generating train2 split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

### Tokenizer

In [7]:
tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

## Creating the dataloader

In [8]:
class token_ds(Dataset):
  def __init__(self, ds, max_length):
    super().__init__()
    length = len(ds)
    max_length += 1
    collector = []
    temp = torch.zeros(1, max_length)

    with tqdm(total=length) as pbar:
      for pos, line in enumerate(ds):
        sentences = line.split('.')
        for sentence in sentences:
          a = torch.tensor(tokenizer.encode(sentence)).long()
          length = a.shape[0]

          if a[1] == 2:
            continue

          if length < max_length:
            a = torch.cat([a, torch.ones(max_length-length)])
          else:
            a = a[-max_length:]

          collector.append(a.unsqueeze(0))
          pos += 1

          # Performance purpose
          if pos%10000 == 0:
            temp = torch.cat([temp, *collector], dim=0).long()
            collector = []

        pbar.update(1)

    temp = torch.cat([temp, *collector], dim=0).long()

    self.value = temp[1:, :-1]
    self.target = temp[1:, 1:]

  def __len__(self):
    return self.value.shape[0]

  def __getitem__(self, i):
    return self.value[i,:], self.target[i, :]

In [9]:
seq_len = 100
batch_size = 32

val_dl = DataLoader(token_ds(dataset['val']['text'], seq_len), num_workers=2, batch_size=batch_size)

100%|██████████| 3760/3760 [00:04<00:00, 772.99it/s]


In [10]:
for pos, (X,Y) in enumerate(val_dl):
  print(f'{X} \n {Y}')
  print(f'{X.shape} \n {Y.shape}')
  break

tensor([[    0,  5457, 11858,  ...,     1,     1,     1],
        [    0, 11858, 42292,  ...,     1,     1,     1],
        [    0,    85,    16,  ...,     1,     1,     1],
        ...,
        [    0,    20,    80,  ...,     1,     1,     1],
        [    0,    20,  4533,  ...,     1,     1,     1],
        [    0, 38187, 40037,  ...,     1,     1,     1]]) 
 tensor([[ 5457, 11858, 42292,  ...,     1,     1,     1],
        [11858, 42292, 20577,  ...,     1,     1,     1],
        [   85,    16,  3615,  ...,     1,     1,     1],
        ...,
        [   20,    80,  4707,  ...,     1,     1,     1],
        [   20,  4533,  6031,  ...,     1,     1,     1],
        [38187, 40037, 10361,  ...,     1,     1,     1]])
torch.Size([32, 100]) 
 torch.Size([32, 100])


## Switching 'Mixture Of Experts' BLock

In [23]:
class SwitchingMoE(nn.Module):
    def __init__(self, d_model, noise_const=0.8, decay_rate=1e-3, dim_feedforward=2048, num_experts=8, dropout=0.1, loss_const=0.01, compute_loss=True):
        super().__init__()
        self.gate = nn.Linear(d_model, num_experts)
        self.num_experts = num_experts
        self.noise_const = noise_const
        self.loss_const = loss_const
        self.decay_rate = decay_rate
        self.compute_loss = compute_loss
        self.FC1 = nn.ModuleList([nn.Linear(d_model, dim_feedforward) for _ in range(num_experts)])
        self.FC2 = nn.ModuleList([nn.Linear(dim_feedforward, d_model) for _ in range(num_experts)])
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.root2 = math.sqrt(2)

    def _compute_aux_loss(self, conditions, values, T):
        f = torch.zeros(self.num_experts, device=device)
        for i in range(self.num_experts):
            f[i] = torch.sum(values == i)
        f = f / T
        p = torch.sum(conditions, dim=0) / T
        return self.loss_const * self.num_experts * torch.sum(f * p)

    def forward(self, x):
        B, T, C = x.shape
        x = x.view(B * T, -1).to(device)
        conditions = F.softmax(self.gate(x), dim=1)
        values = torch.argmax(conditions, dim=1)

        if self.compute_loss and self.training:
            aux_loss = self._compute_aux_loss(conditions, values, B * T)
            self.aux_loss = aux_loss
        else:
            self.aux_loss = 0

        for i in range(self.num_experts):
            mask = values == i
            if torch.any(mask):
                x[mask] = self.dropout2(self.FC2[i](self.dropout1(F.relu(self.FC1[i](x[mask])))))

        x = x.view(B, T, C)
        return x

## Creating the model

In [24]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, noise_const=0.8, dim_feedforward=2048, num_experts=8, dropout=0.1, loss_const=0.01, compute_loss=True):
        super().__init__()
        if d_model != 512 and dim_feedforward == 2048:
            dim_feedforward = 4 * d_model

        self.self_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, batch_first=True)
        self.switching_moe = SwitchingMoE(
            d_model=d_model,
            noise_const=noise_const,
            dim_feedforward=dim_feedforward,
            num_experts=num_experts,
            dropout=dropout,
            loss_const=loss_const,
            compute_loss=compute_loss,
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        x = self.norm1(x + self._sa_block(x, src_mask))
        x = self.norm2(x + self._ff_block(x))
        return x

    def _sa_block(self, x, mask=None):
        return self.dropout(self.self_attn(x, x, x, attn_mask=mask)[0])

    def _ff_block(self, x):
        return self.switching_moe(x)

In [34]:
class TransformerModel(nn.Module):
    def __init__(self, ntokens, d_model, nhead, seq_len, device, noise_const=0.8, nlayers=6, num_experts=8, dim_feedforward=2048, dropout=0.1, compute_loss=True):
        super(TransformerModel, self).__init__()
        """
        ntokens: The number of words in the dictionary
        d_model: the size of each word embedding
        n_head: the number of heads in each encoder layer
        nlayers: the number of transformer encoder layers in the encoder
        """
        self.d_model = d_model
        self.device = device
        self.seq_len = seq_len

        self.embed = nn.Embedding(ntokens, d_model, padding_idx=0)
        self.pos_embed = PositionalEncoding1D(d_model)

        self.encoder = nn.ModuleList([
            TransformerEncoderLayer(
                d_model,
                nhead,
                noise_const=noise_const,
                num_experts=num_experts,
                dim_feedforward=dim_feedforward,
                dropout=dropout,
                compute_loss=compute_loss
            ) for _ in range(nlayers)
        ])

        self.fc = nn.Linear(d_model, ntokens)
        self.dropout = nn.Dropout(dropout)
        self.src_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(device)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embed.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        self.fc.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None):
        src = self.embed(src) * math.sqrt(self.d_model)
        src = src + self.pos_embed(src)

        if src_mask is None:
            src_mask = self.src_mask

        for encoder in self.encoder:
            src = encoder(src, src_mask=src_mask)

        logits = self.fc(self.dropout(src))
        output = F.log_softmax(logits, dim=-1)
        return output

### Sequence generator

In [35]:
"""def generate(src: str, num_words: int):
  model.eval()
  src = torch.tensor(tokenizer.encode(src)[:-1]).long()

  for _ in range(num_words):
    # Pad the input sequence properly so that the model can understand
    temp = src
    pos = seq_len-1
    if int(src.shape[0]) < seq_len:
      temp = torch.cat([src, torch.ones(seq_len-src.shape[0], dtype=torch.long)])
      pos = src.shape[0]
    if int(src.shape[0]) > seq_len:
      temp = src[-seq_len:]

    # Get the predictions of words from the model
    temp = temp.unsqueeze(0).to(device)
    dist = model(temp) # (1, seq_len, word_probability)
    dist = dist.squeeze(0)[pos-1]  # <----- (pos-1)

    # Sort the words in most likely to come next
    dist = torch.sort(dist)[1]

    # Pick out the word from the top 3 recommendations of the model
    value = int(torch.rand(1)*3)
    token = dist[-(1+value)].unsqueeze(dim=0).to('cpu')

    # Add that word to the sentence
    src = torch.cat([src, token])

    if token == 2:
      continue

  src = tokenizer.decode(list(src))
  model.train()

  return src"""

In [41]:
def generate(src: str, num_words: int):
    model.eval()
    try:
        src = torch.tensor(tokenizer.encode(src)[:-1]).long().to(device)

        for _ in range(num_words):
            temp = src
            pos = min(src.shape[0], seq_len) - 1
            if src.shape[0] < seq_len:
                temp = torch.cat([src, torch.ones(seq_len - src.shape[0], dtype=torch.long, device=device)])
            elif src.shape[0] > seq_len:
                temp = src[-seq_len:]

            temp = temp.unsqueeze(0).to(device)
            dist = model(temp)
            dist = dist.squeeze(0)[pos]

            dist = torch.sort(dist, descending=True)[1]
            value = int(torch.rand(1).item() * 3)
            token = dist[value].unsqueeze(dim=0)

            src = torch.cat([src, token])

            if token.item() == 2:
                break

        src = tokenizer.decode(list(src.cpu().numpy()))
    finally:
        model.train()

    return src

# Training function

In [36]:
loss_fn = nn.CrossEntropyLoss()

In [37]:
def train(model, optim, train_dl, val_dl, l_fn, epochs, check_overfit=True):
  H = {
      'train_loss': [],
      'train_acc': [],
      'val_loss': [],
      'val_acc': [],
  }
  e0 = 0

  for e in range(epochs):
    model.train()

    print(f"------------------------------------------------------------")
    print(f"EPOCH : {e+1}")

    train_loss = 0
    val_loss = 0
    train_correct = 0
    val_correct = 0

    print("Training Step:")
    with tqdm(total=len(train_dl)) as pbar:
      for _, (X, Y) in enumerate(train_dl):
        Y = Y.view(-1)
        X, Y = (X.to(device), Y.to(device))

        # Forward prop
        pred = model(X)
        B,T,C = pred.shape
        pred = pred.view(B*T, C)
        loss = loss_fn(pred, Y)
        train_loss += loss.to("cpu").detach().numpy()

        # Back prop
        optim.zero_grad()
        loss.backward()
        optim.step()
        train_correct += sum([1 for i, val in enumerate(pred.argmax(1)) if int(val) == int(Y[i])])

        pbar.update(1)

    print("\nValidation Step:")
    with tqdm(total=len(val_dl)) as pbar:
      with torch.no_grad():
        model.eval()

        for _, (X, Y) in enumerate(val_dl):
          Y = Y.view(-1)
          X, Y = (X.to(device), Y.to(device))

          pred = model(X)
          B,T,C = pred.shape
          pred = pred.view(B*T, C)
          loss = loss_fn(pred, Y)

          val_loss += loss.to("cpu").detach().numpy()
          val_correct += sum([1 for i, val in enumerate(pred.argmax(1)) if int(val) == int(Y[i])])
          pbar.update(1)


    # Store the important specifications of the training process
    H["train_loss"].append(train_loss)
    H["train_acc"].append(train_correct / len(train_dl.dataset))
    H["val_loss"].append(val_loss)
    H["val_acc"].append(val_correct/ len(val_dl.dataset))

    # Print the messages appropriately
    print(f"\n\nTrain Loss : {train_loss:.2f}")
    print(f"Val Loss : {val_loss:.2f}")
    print(f"Train Accuracy : {train_correct/len(train_dl.dataset):.4f}")
    print(f"Val Accuracy : {val_correct/len(val_dl.dataset):.4f}")

    # Check for overfitting
    if check_overfit and (len(H['val_acc']) > 1 and H['val_acc'][-1] < H['val_acc'][-2]):
      print("The model is showing signs of over fitting enter Y to continue, or N for breaking the training loop")
      t = input()
      if t == 'Y':
        continue
      else:
        return H, e
    e0 = e
  return H, e0

# Training the model

In [38]:
# Hyperparameters
nhead = 4
d_model = 256
ntokens = tokenizer.vocab_size
num_inputs = seq_len
model = TransformerModel(nhead=nhead,
                         d_model=d_model,
                         ntokens=ntokens,
                         num_experts=4,
                         seq_len=num_inputs,
                         nlayers=4,
                         device=device,
                         compute_loss=True,
                         noise_const=0.4).to(device)

opt = torch.optim.Adam(model.parameters(), lr=3e-5)

In [39]:
epochs = 3
num_sequences = 10_000
num_repetitions = 5

for i in range(num_repetitions):
  train_dl = DataLoader(token_ds(train_ds[i*num_sequences:(i+1)*num_sequences], seq_len), num_workers=2, batch_size=batch_size)

  start_time = time.time()
  train(model, opt, train_dl, val_dl, loss_fn, epochs, check_overfit=True)
  end_time = time.time()

  print(f'\nElapsed Time: {end_time-start_time} \n')

100%|██████████| 10000/10000 [00:10<00:00, 965.58it/s]


------------------------------------------------------------
EPOCH : 1
Training Step:


100%|██████████| 901/901 [03:53<00:00,  3.85it/s]



Validation Step:


100%|██████████| 352/352 [00:51<00:00,  6.90it/s]


...



Train Loss : 1211.00
Val Loss : 477.95
Train Accuracy : 82.0057
Val Accuracy : 82.3848
------------------------------------------------------------
EPOCH : 3
Training Step:


100%|██████████| 873/873 [03:47<00:00,  3.84it/s]



Validation Step:


100%|██████████| 352/352 [00:53<00:00,  6.61it/s]



Train Loss : 1192.37
Val Loss : 476.89
Train Accuracy : 82.0956
Val Accuracy : 82.4109

Elapsed Time: 828.1723756790161 



# Testing

In [50]:
test = 'from what i understand'
generate(test, num_words=500)

'<s>from what i understand the album and was a number the game , and the same , but the city was a number is the United Kingdom of his first of his father and a number </s>'